## Assignment 2 - LM 

In [195]:
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [196]:
r=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text)


In [197]:
# table=soup.selec("table",attrs={'class': 'wikitable sortable'})
trs = soup.find('table',attrs={'class': 'wikitable sortable'}).find_all('tr')


In [198]:
column_names =["PostalCode", "Borough", "Neighborhood"]
df=pd.DataFrame(columns=column_names)
for tr in trs:
    l=[]
    for td in tr.find_all("td"):
        s=td.text.strip()
        l.append(s)
    if l:
        if l[1]=="Not assigned":pass
        else:
            if l[2]=="Not assigned": l[2]=l[1]
            df=df.append({"PostalCode":l[0],"Borough":l[1],"Neighborhood":l[2]},ignore_index=True)
df.head()    

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [199]:
post=df['PostalCode'].unique()
df2=pd.DataFrame(columns=column_names)
for p in post:
    b=df.loc[df['PostalCode']==p]["Borough"].values[0]
    n=",".join(df.loc[df['PostalCode']==p]["Neighborhood"].values)
    df2=df2.append({"PostalCode":p,"Borough":b,"Neighborhood":n},ignore_index=True)
print(df2.head(10))
df2.shape

  PostalCode           Borough                     Neighborhood
0        M3A        North York                        Parkwoods
1        M4A        North York                 Victoria Village
2        M5A  Downtown Toronto         Harbourfront,Regent Park
3        M6A        North York  Lawrence Heights,Lawrence Manor
4        M7A      Queen's Park                     Queen's Park
5        M9A         Etobicoke                 Islington Avenue
6        M1B       Scarborough                    Rouge,Malvern
7        M3B        North York                  Don Mills North
8        M4B         East York   Woodbine Gardens,Parkview Hill
9        M5B  Downtown Toronto          Ryerson,Garden District


(103, 3)

In [201]:
print('The dataframe has {} PostalCode .'.format(
        len(df2['PostalCode'].unique()),
    )
)

The dataframe has 103 PostalCode .


In [213]:
df3=df2.loc[df2['Borough'].str.contains("Toronto")].reset_index(drop=True)
df3

,PostalCode,Borough,Neighborhood
0,M5A,Downtown Toronto,"Harbourfront,Regent Park"
1,M5B,Downtown Toronto,"Ryerson,Garden District"
2,M5C,Downtown Toronto,St. James Town
3,M4E,East Toronto,The Beaches
4,M5E,Downtown Toronto,Berczy Park
5,M5G,Downtown Toronto,Central Bay Street
6,M6G,Downtown Toronto,Christie
7,M5H,Downtown Toronto,"Adelaide,King,Richmond"
8,M6H,West Toronto,"Dovercourt Village,Dufferin"
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station"


In [209]:
df_ll=pd.read_csv('http://cocl.us/Geospatial_data')

In [210]:
df_ll.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
